In [1]:
%matplotlib widget

In [2]:
import numpy as np
import pandas as pd
import random

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

In [3]:
data = pd.read_hdf('../feature_exploration/features.h5', key='no_preprocessing')

In [4]:
# get the subjects for which LOSO actually makes sense: those with multiple activities (ie more than just walking)
gbc = data.groupby(['Subject', 'Activity'], as_index=False).count()
loso_subjects = [i for i in gbc.Subject.unique() if gbc.loc[gbc.Subject == i].shape[0] > 3]

random.seed(5)  # fix the generation so that its the same every time
random.shuffle(loso_subjects)

training_masks = []
validation_masks = []
testing_masks = []

for i in range(0, len(loso_subjects), 3):
    tr_m = np.ones(data.shape[0], dtype='bool')
    v_m = np.zeros(data.shape[0], dtype='bool')
    
    for j in range(3):
        tr_m &= data.Subject != loso_subjects[i+j]
    for j in range(2):
        v_m |= data.Subject == loso_subjects[i+j]
    te_m = data.Subject == loso_subjects[i+2]
    
    training_masks.append(tr_m)
    validation_masks.append(v_m)
    testing_masks.append(te_m)

masks = (training_masks, validation_masks, testing_masks)

In [5]:
def one_hot2(x):
    res = np.zeros((x.size, 2), dtype='long')
    res[x == 0, 0] = 1
    res[x == 1, 1] = 1
    return res

In [6]:
X_train = torch.tensor(
    data.drop(
        ['Subject', 'Activity', 'Label'], 
        axis=1
    ).loc[training_masks[0]].values,
    dtype=torch.float,
    device=torch.device('cuda:0')
)
Y_train = torch.tensor(
    data.loc[training_masks[0], 'Label'].values,
    dtype=torch.long,
    device=torch.device('cuda:0')
)

# validation set
X_val = torch.tensor(
    data.drop(
        ['Subject', 'Activity', 'Label'], 
        axis=1
    ).loc[validation_masks[0]].values,
    dtype=torch.float,
    device=torch.device('cuda:0')
)
Y_val = torch.tensor(
    data.loc[validation_masks[0], 'Label'].values,
    dtype=torch.long,
    device=torch.device('cuda:0')
)

train_ds = TensorDataset(X_train, Y_train)
val_ds = TensorDataset(X_val, Y_val)

In [7]:
class Net(nn.Module):
    def __init__(self, n_feats):
        super().__init__()
        
        self.fc1 = nn.Linear(n_feats, 32)
        self.fc2 = nn.Linear(32, 8)
        self.fc3 = nn.Linear(8, 2)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x
    
    def predict(self, x):
        pred_ = self.forward(x)
        pred = []
        for p in pred_:
            if p[1] > p[0]:
                pred.append(1)
            else:
                pred.append(0)
        return np.array(pred)

In [8]:
model = Net(26).cuda()

crit = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [53]:
train_dl = DataLoader(train_ds, batch_size=128)

for c in range(8):
    tloss = 0
    for xb, yb in train_dl:
        y_pred = model.forward(xb)

        loss = crit(y_pred, yb)
        tloss += loss.item()

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()
    print(f'Epoch ({c+1}): loss: {tloss / X_train.shape[0]:.4f}')

Epoch (1): loss: 0.0048
Epoch (2): loss: 0.0048
Epoch (3): loss: 0.0048
Epoch (4): loss: 0.0048
Epoch (5): loss: 0.0048
Epoch (6): loss: 0.0048
Epoch (7): loss: 0.0048
Epoch (8): loss: 0.0048


In [47]:
xb, y_true = val_ds[:]
y_pred = model.predict(xb)

In [48]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(y_pred, label='Prediction')
plt.plot(y_true.cpu().detach().numpy(), label='Truth')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …